# Test code of GCTF Function

## Dependent Packages Installation

In [ ]:
if (!requireNamespace("rTensor", quietly = TRUE)){
    install.packages("rTensor")
}
if (!requireNamespace("testthat", quietly = TRUE)){
    install.packages("testthat")
}

library("rTensor")
library("testthat")

## GCTF Function (Paste here)

In [ ]:
GCTF <- function(){
    
}

## Simulation Datasets

In [ ]:
# I × J × K
X1 <- rand_tensor(modes = c(4,5,6))
X1 <- X1@data^2
names(dim(X1)) <- c("I", "J", "K")

# I × P
X2 <- matrix(runif(4*7), nrow=4, ncol=7)
names(dim(X2)) <- c("I", "M")

# J × Q
X3 <- matrix(runif(5*8), nrow=5, ncol=8)
names(dim(X3)) <- c("J", "N")

# Coupled Tensor/Matrix
X <- list(X1 = X1, X2 = X2, X3 = X3)

# Coupling matrix R（CP）
R_CP <- rbind(
    c(1,1,1,0,0),
    c(1,0,0,1,0),
    c(0,1,0,0,1)
)
rownames(R_CP) <- paste0("X", seq(3))
colnames(R_CP) <- LETTERS[seq(5)]

# Size of Factor matrices (CP)
Ranks_CP <- list(
    A=list(I=4, r=3),
    B=list(J=5, r=3),
    C=list(K=6, r=3),
    D=list(M=7, r=3),
    E=list(N=8, r=3))

# Coupling matrix R（Tucker）
R_Tucker <- rbind(
    c(1,1,1,1,0,0),
    c(1,0,0,0,1,0),
    c(0,1,0,0,0,1)
)
rownames(R_Tucker) <- paste0("X", seq(3))
colnames(R_Tucker) <- LETTERS[seq(6)]

# Size of Factor matrices (Tucker)
Ranks_Tucker <- list(
    A=list(I=4, p=3),
    B=list(J=5, q=4),
    C=list(K=6, r=5),
    D=list(p=3, q=4, r=5),
    E=list(M=7, p=3),
    F=list(N=8, q=4))

## Perform GCTF against Simulation Datasets

In [ ]:
# CP
out.CP_EUC <- GCTF(X, R_CP, Ranks=Ranks_CP, Beta=0)
out.CP_KL <- GCTF(X, R_CP, Ranks=Ranks_CP, Beta=1)
out.CP_IS <- GCTF(X, R_CP, Ranks=Ranks_CP, Beta=2)

# Tucker
out.CP_EUC <- GCTF(X, R_Tucker, Ranks=Ranks_Tucker, Beta=0)
out.Tucker_KL <- GCTF(X, R_Tucker, Ranks=Ranks_Tucker, Beta=1)
out.Tucker_IS <- GCTF(X, R_Tucker, Ranks=Ranks_Tucker, Beta=2)

## Unit Tests (Synthetic datasets)

In [ ]:
expect_identical()

## Real Datasets

## Perform GCTF against Real Datasets

## Reproduce the result of original paper